# imports

In [221]:
import os, sys

import numpy as np, cPickle as pickle, copy, string

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.image as mpimg

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.ticker import MaxNLocator, FormatStrFormatter

# matplotlib settings

In [350]:
##############################################################################################
# PARAM LIST AND TEST CASE ADAPATED FROM WORK BY Ben Barad, Fraser Lab, UCSF
# ONLINE SOURCE: https://github.com/bbarad/matplotlibrc
##############################################################################################

PLOTFMT = 'svg'
SAVEDIR = os.path.abspath('./final_figs')

# Fig widths as specified by JCP
col_single = 8.5 #cm
col_dbl = 17.0 #cm

# Matplotlib paramlist
PARAMS =  {
        'path.simplify': True,

        # Backends
        'backend': 'agg',

        # Fonts
        'font.family' : 'sans serif',
        #'font.serif': '["Computer Modern"]',
        'font.style': 'normal',
        'font.variant': 'normal',
        'font.weight' : 'medium',
        'font.stretch': 'normal',
        'font.size': 14.0,

        # text
        'text.color': 'black',
        'text.usetex': True,
        'mathtext.default': 'regular',

        # Axes
        'axes.facecolor' : 'white',
        'axes.edgecolor': 'black',
        'axes.formatter.use_mathtext' : 'True',
        'axes.titlesize': 14.0,
        'axes.labelsize': 14.0,
        'axes.xmargin': 0,
        'axes.ymargin': 0,
        #'axes.labelpad': 1.0,
        'xtick.major.size' : 4,
        'ytick.major.size' : 4,
        'xtick.minor.size' : 2,
        'ytick.minor.size' : 2,
        'xtick.major.width' : 1,
        'ytick.major.width' : 1,
        'xtick.minor.width' : 1,
        'ytick.minor.width' : 1,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,

        # Legends
        'legend.numpoints' : 1,
        'legend.frameon' : False,
        'legend.handletextpad' : 0.3,
        'legend.fontsize': 14.0,
        'legend.loc': 'best',
        'legend.markerscale' : 1,
        'legend.labelspacing': 0.5,

        # Figures
        #'figure.autolayout': True,
        'figure.facecolor' :'w',
        'figure.edgecolor' :'w',
        'figure.dpi': (300),
        'savefig.dpi': (300),
        'savefig.facecolor' : 'w',
        'savefig.edgecolor': 'w',
        'savefig.format': PLOTFMT,
        'savefig.bbox': 'tight',
        'savefig.pad_inches': 0.04,

        # Lines
        'lines.linestyle': 'solid',
        'lines.linewidth': 3,
        'lines.markersize': 6,
        'lines.color': 'black',
        'lines.solid_capstyle': 'round',
        'lines.solid_joinstyle': 'round'

        }

# apply settings
matplotlib.rcParams.update(PARAMS)


def set_width(col_type = 'single'):
        if col_type == 'single': return col_single/2.54
        if col_type == 'dbl': return col_dbl/2.54

def prune(ax = None, nbins = 5, doX = True, doY = True):
    if ax is None: return
    if doX: ax.xaxis.set_major_locator(MaxNLocator(nbins = nbins, prune = 'both'))
    if doY: ax.yaxis.set_major_locator(MaxNLocator(nbins = nbins, prune = 'both'))

def tight_layout():
    # manual tight_layout for situations
    # when the built-in fails
    plt.subplots_adjust(left = 0.18, bottom = 0.18)

def fixSize(fig, width_type = 'single'):
    figsize_0 = fig.get_size_inches()
    w0, h0 = figsize_0[0], figsize_0[1]
    if width_type == 'single': w1 = col_single
    else: w1 = col_dbl
    h1 = w1 * (h0/w0)
    fig.set_size_inches([w1, h1])

def savefig(fig, Prefix = None):
    if Prefix:
        OutPrefix = os.path.join(SAVEDIR, Prefix)
        fig.savefig(OutPrefix + '.' + PLOTFMT, bbox_inches = 'tight')
    return

# utility functions and constants

In [223]:
# boltzmann constant
kB = 0.001987

# target peptides
small_set = ['1cb3', '1l2y', '2i9m', 'c_pep', 'ek_pep', '15beta', '1e0q', '1gb1', '1hrx', '1j4m']
large_set = ['1bdd', '2fs1', '2a3d', '1e0n', '1e0l', '1ubq', '1pgb', '1shg', '1srl']
small_set_labels = ['1CB3', '1L2Y', '2I9M', 'C Peptide', 'EK Peptide', r'$15-\beta$', '1E0Q', '1GB1', '1LE1', '1J4M']
large_set_labels = ['Protein A', 'Albumin-binding domain protein', r'$\alpha 3 \mathrm{D}$', 'YJQ8 WW domain(res 7-31)', 
                    'FBP28 WW domain (res 6-31)', 'Ubiquitin fragment (res 1-35)', 'Protein G', 
                    r'$\alpha-$' + ' spectrin SH3', 'src-SH3']

# forcefields
fftypes = ['leu15', 'val15', 'leu_val_heteropoly_360', 'leu_val_ex_300']
ffclrs = ['lightcoral', 'lightskyblue', 'yellowgreen', 'goldenrod']
fflabels = [r'$\mathrm{LEU}_{15}$', r'$\mathrm{VAL}_{15}$',
            r'$\mathrm{LEU}_6 \mathrm{VAL}_9$', r'$\mathrm{LEU}_{15} + \mathrm{VAL}_{15}$']

# find temp nearest to T in the replica temps TList
def NearestTemp(T, TList):
    return TList[np.argmin(abs(TList - T))]

# trim a 2D pmf to:
# a) clip all finite values to Emax (~ 5 kT)
# b) set the avg of last n bins to 0
def TrimPmf(pmf, Emax = 15, n = 10):
    for i in range(pmf.shape[0]):
        for j in range(pmf.shape[1]):
            if np.isfinite(pmf[i,j]) and pmf[i,j] > Emax:
                pmf[i,j] = Emax
    pmf_ = pmf[np.isfinite(pmf)]
    pmf_ = pmf_.flatten()
    offset = np.mean(np.sort(pmf_)[0:n])
    pmf -= offset
    pmf -= np.min(pmf)
    return pmf

# plot a pmf after transforming it appropriately
def plot_pmf(ax, pmf_data, func = 'contourf', Emax = None, n = None):
    (x,y), pmf, err = pmf_data
    if Emax is None: Emax = 15
    if n is None: n = 10
    pmf = TrimPmf(pmf, Emax = Emax, n = n)
    pmf = np.transpose(pmf)
    if func == 'contourf': plotfunc = ax.contourf
    if func == 'imshow': plotfunc = ax.imshow
    im = plotfunc(pmf, origin = 'lower', aspect = 'auto', interpolation = 'none',
                  extent = [x.min(), x.max(), y.min(), y.max()], cmap = cm.jet)
    return im

# CG polypeptides

#### leu-15 and val-15 folding curves

In [239]:
fig, axs = plt.subplots(1,2, figsize = (set_width('dbl'), 3))
axs = axs.flat

DIR = 'DATA/CGPOLYPEP'

# leu-15 folding curves
ax1 = axs[0]
with open(os.path.join(DIR, 'leu15', 'aa_foldcurve.pickle'), 'r') as of: t_aa, f_aa, e_aa = pickle.load(of)
with open(os.path.join(DIR, 'leu15', 'cg_foldcurve.pickle'), 'r') as of: t_cg, f_cg, e_cg = pickle.load(of)
ax1.plot(t_aa, f_aa, lw = 2, color = 'blue', label = 'AA')
ax1.fill_between(t_aa, f_aa-e_aa, f_aa+e_aa, color = 'lightskyblue', alpha = 0.4)
ax1.plot(t_cg, f_cg, lw = 2, color = 'red', label = 'CG')
ax1.fill_between(t_cg, f_cg-e_cg, f_cg+e_cg, color = 'lightcoral', alpha = 0.4)
ax1.axvline(407., ls = '--', lw = 2)
ax1.legend()
ax1.set_ylabel('folding fraction')
ax1.set_title('polyleucine ' + r'$(\alpha-\mathrm{helix})$')

# val-15 folding curves
ax2 = axs[1]
with open(os.path.join(DIR, 'val15', 'aa_foldcurve.pickle'), 'r') as of: t_aa, f_aa, e_aa = pickle.load(of)
with open(os.path.join(DIR, 'val15', 'cg_foldcurve.pickle'), 'r') as of: t_cg, f_cg, e_cg = pickle.load(of)
ax2.plot(t_aa, f_aa, lw = 2, color = 'blue', label = 'AA')
ax2.fill_between(t_aa, f_aa-e_aa, f_aa+e_aa, color = 'lightskyblue', alpha = 0.4)
ax2.plot(t_cg, f_cg, lw = 2, color = 'red', label = 'CG')
ax2.fill_between(t_cg, f_cg-e_cg, f_cg+e_cg, color = 'lightcoral', alpha = 0.4)
ax2.axvline(367., ls = '--', lw = 2)
ax2.set_yticklabels([])
ax2.set_title('polyvaline ' + r'$(\beta-\mathrm{hairpin})$')

for ax in [ax1, ax2]:
    ax.set_xlim([270., 500.])
    ax.set_ylim([0., 1.])
    prune(ax, nbins = 6)
    ax.set_xlabel('temp (K)')
    
# put panel labels
for n, ax in enumerate(axs):
    txt = '(%s)' % string.ascii_uppercase[n]
    ax.text(0, 1.05, txt, transform = ax.transAxes, weight = 'bold')

plt.subplots_adjust(wspace = 0.0, hspace = 0.0)  
savefig(fig, 'fig3')
plt.close()

#### use new matplotlibrc settings for figs 4 - 6

In [351]:
newparams = {'axes.titlesize': 15.0,
             'axes.labelsize': 15.0,
             'legend.fontsize': 15.0,
             'font.size': 15.0,
             'xtick.labelsize': 15,
             'ytick.labelsize': 15}
for k,v in newparams.iteritems():
    matplotlib.rcParams[k] = v

#### leu-15 and val-15 free energy surfaces

In [352]:
fig, axs = plt.subplots(nrows = 2, ncols = 2, figsize = (set_width('dbl'), 6))
axs = axs.flat
DIR = 'DATA/CGPOLYPEP'

# leu-15 pmfs
ax1 = axs[0]
ax2 = axs[1]
with open(os.path.join(DIR, 'leu15', 'aa.280_pmf.pickle'), 'r') as of: data_aa = pickle.load(of)
with open(os.path.join(DIR, 'leu15', 'cg.280_pmf.pickle'), 'r') as of: data_cg = pickle.load(of)
im1 = plot_pmf(ax1, data_aa, Emax = 15., n = 10)
im2 = plot_pmf(ax2, data_cg, Emax = 15., n = 10)
ax1.set_xticklabels([])
ax1.set_ylabel(r'$\mathrm{RMSD}_{\alpha-\mathrm{helix}} (\mathring{A})$')
ax1.set_title('polyleucine AA')
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax2.set_title('polyleucine CG')

# val-15 pmfs
ax3 = axs[2]
ax4 = axs[3]
with open(os.path.join(DIR, 'val15', 'aa.280_pmf.pickle'), 'r') as of: data_aa = pickle.load(of)
with open(os.path.join(DIR, 'val15', 'cg.280_pmf.pickle'), 'r') as of: data_cg = pickle.load(of)
im3 = plot_pmf(ax3, data_aa, Emax = 15., n = 10)
im4 = plot_pmf(ax4, data_cg, Emax = 15., n = 10)
ax3.set_ylabel(r'$\mathrm{RMSD}_{\beta-\mathrm{hairpin}} (\mathring{A})$')
ax3.set_title('polyvaline AA')
ax4.set_yticklabels([])
ax4.set_title('polyvaline CG')

for ax in [ax1, ax2, ax3, ax4]:
    ax.set_xlim([5., 12.])
    ax.set_ylim([0., 10.])
    prune(ax)
    if ax == ax3 or ax == ax4:
         ax.set_xlabel(r'$R_g (\mathring{A})$')

plt.subplots_adjust(wspace = 0.)

# colorbar
cax = fig.add_axes([0.91, 0.125, 0.02, 0.755])
cbar = fig.colorbar(im1, cax=cax)
cbar.set_label(r'$\frac{\Delta F}{k_B T}$', size = 25, family = 'sans serif', 
               rotation = 'vertical', labelpad = 10)

# put panel labels
ax1.text(0, 1.05, 'A', transform = ax1.transAxes, weight = 'bold')
ax3.text(0, 1.05, 'B', transform = ax3.transAxes, weight = 'bold')
    
savefig(fig, 'fig4')
plt.close()

#### leu_val heteropolymer @ 360K

In [346]:
DIR = 'DATA/CGPOLYPEP_final_2/leu_val_heteropoly_360'

fig, axs = plt.subplots(nrows = 2, ncols = 2, figsize = (set_width('dbl') * 1.2, 8))
axs = axs.flat

# alpha folding curves
ax1 = axs[0]
with open(os.path.join(DIR, 'aa_STRIDE.pickle'), 'r') as of:
    ss_aa = pickle.load(of)
    t, ss_h_aa, e_ss_h_aa = ss_aa[0]
    t, ss_b_aa, e_ss_b_aa = ss_aa[1]
with open(os.path.join(DIR, 'cg_STRIDE.pickle'), 'r') as of:
    ss_cg = pickle.load(of)
    t, ss_h_cg, e_ss_h_cg = ss_cg[0]
    t, ss_b_cg, e_ss_b_cg = ss_cg[1]
with open(os.path.join(DIR, 'alpha_consistent', 'aa_foldcurve.pickle'), 'r') as of: t, f_h_aa, e_h_aa = pickle.load(of)
with open(os.path.join(DIR, 'alpha_consistent', 'cg_foldcurve.pickle'), 'r') as of: t, f_h_cg, e_h_cg = pickle.load(of)
ax1.plot(t, f_h_aa, lw = 2, color = 'blue', label = 'AA')
ax1.plot(t, f_h_cg, lw = 2, color = 'red', label = 'CG')
ax1.fill_between(t, f_h_aa-e_h_aa, f_h_aa+e_h_aa, color = 'lightskyblue', alpha = 0.4)
ax1.fill_between(t, f_h_cg-e_h_cg, f_h_cg+e_h_cg, color = 'lightcoral', alpha = 0.4)
ax1.legend()
ax1.set_title(r'$\alpha-$' + ' helix fraction')
ax1.set_ylabel('folding fraction')
ax1.axvline(360., ls = '--', lw = 2)

#beta folding curves
ax2 = axs[1]
with open(os.path.join(DIR, 'beta_consistent', 'aa_foldcurve.pickle'), 'r') as of: t, f_b_aa, e_b_aa = pickle.load(of)
with open(os.path.join(DIR, 'beta_consistent', 'cg_foldcurve.pickle'), 'r') as of: t, f_b_cg, e_b_cg = pickle.load(of)
ax2.plot(t, f_b_aa, lw = 2, color = 'blue')
ax2.plot(t, f_b_cg, lw = 2, color = 'red')
ax2.fill_between(t, f_b_aa-e_b_aa, f_b_aa+e_b_aa, color = 'lightskyblue', alpha = 0.4)
ax2.fill_between(t, f_b_cg-e_b_cg, f_b_cg+e_b_cg, color = 'lightcoral', alpha = 0.4)
ax2.set_title(r'$\beta-$' + ' hairpin fraction')
ax2.set_yticks([])
ax2.set_yticklabels([])

for ax in [ax1, ax2]:
    ax.set_xlabel('temp (K)')
    ax.set_xlim([260., 500.])
    ax.set_ylim([0., 1.])
    prune(ax, nbins = 6)
    ax.axvline(360., ls = '--', lw = 2)

# pmf
ax3 = axs[2]
ax4 = axs[3]
with open(os.path.join(DIR, 'alpha', 'aa.280_pmf.pickle'), 'r') as of: data_aa = pickle.load(of)
with open(os.path.join(DIR, 'alpha', 'cg.280_pmf.pickle'), 'r') as of: data_cg = pickle.load(of)
im3 = plot_pmf(ax3, data_aa, Emax = 15., n = 10)
im4 = plot_pmf(ax4, data_cg, Emax = 15., n = 10)
ax3.set_ylabel(r'$\mathrm{RMSD}_{\alpha-helix} (\mathring{A})$')
ax3.set_title('AA')
ax4.set_title('CG')
ax4.set_yticks([])
ax4.set_yticklabels([])
for ax in [ax3, ax4]:
    ax.set_xlabel(r'$R_g (\mathring{A})$')
    ax.set_xlim([5.5, 14])
    ax.set_ylim([0., 9.5])
    prune(ax)

# colorbar
cax = fig.add_axes([0.91, 0.12, 0.015, 0.33])
cbar = fig.colorbar(im2, cax=cax)
cbar.set_label(r'$\frac{\Delta F}{k_B T}$', size = 25, family = 'sans serif', 
               rotation = 'vertical', labelpad = 10)

# put panel labels
ax1.text(0, 1.05, 'A', transform = ax1.transAxes, weight = 'bold')
ax3.text(0, 1.05, 'B', transform = ax3.transAxes, weight = 'bold')

plt.subplots_adjust(wspace = 0, hspace = 0.35)
savefig(fig, 'fig5')
plt.close()

#### leu_val extended ensemble (@ 300 K) (best so far) folding curves, free energies and ramachandran plots

In [331]:
DIR = 'DATA/CGPOLYPEP/leu_val_ex_300'

fig, axs = plt.subplots(nrows = 2, ncols = 2, figsize = (set_width('dbl') * 1.2, 7))
axs = axs.flat

# pmf
ax1 = axs[0]
with open(os.path.join(DIR, 'alpha', 'cg.280_pmf.pickle'), 'r') as of: data = pickle.load(of)
im1 = plot_pmf(ax1, data, Emax = 15., n = 10)
ax1.set_xlabel(r'$R_g (\mathring{A})$')
ax1.set_ylabel(r'$\mathrm{RMSD}_{\alpha-helix} (\mathring{A})$')
ax1.set_xlim([5.5, 14])
ax1.set_ylim([0., 9.5])
prune(ax1)

# ramachandran plot
ax2 = axs[1]
with open(os.path.join(DIR, 'cg.280_rama.pickle'), 'r') as of: phi, psi, data = pickle.load(of)
(x,y), pmf, err  = data
x *= 180. / np.pi
y *= 180. / np.pi
pmf = -np.log(pmf)
data = (x,y), pmf, err
im2 = plot_pmf(ax2, data, Emax = 15., n = 10)
ax2.set_xlabel(r'$\varphi (^\circ)$')
ax2.set_ylabel(r'$\psi (^\circ)$', labelpad = -20)
ax2.set_xlim([-180., 180.])
ax2.set_ylim([-180., 180.])
ax2.grid('on', color = 'k', lw = 1)
ax2.axhline(0., ls = 'solid', lw = 1, color = 'black')
ax2.axvline(0., ls = 'solid', lw = 1, color = 'black')
prune(ax2)
# colorbar
cax = fig.add_axes([0.99, 0.6, 0.015, 0.37])
cbar = fig.colorbar(im2, cax=cax)
cbar.set_label(r'$\frac{\Delta F}{k_B T}$', size = 25, family = 'sans serif', 
               rotation = 'vertical', labelpad = 10)

# folding curves
ax3 = axs[2]
with open(os.path.join(DIR, 'alpha', 'cg_foldcurve.pickle'), 'r') as of: t1, f1, e1 = pickle.load(of)
with open(os.path.join(DIR, 'beta', 'cg_foldcurve.pickle'), 'r') as of: t2, f2, e2 = pickle.load(of)
with open(os.path.join(DIR, 'cg_STRIDE.pickle'), 'r') as of: ss = pickle.load(of)
t, f_h, e_h = ss[0]
t, f_b, e_b = ss[1]
ax3.plot(t1, f1, lw = 2, color = 'darkorchid', label = r'$\alpha-\mathrm{helix} $')
ax3.plot(t1, f_h, ls = '--', lw = 2, color = 'darkorchid')
ax3.plot(t2, f2, lw = 2, color = 'goldenrod', label = r'$\beta-\mathrm{hairpin}$')
ax3.plot(t2, f_b, ls = '--', lw = 2, color = 'goldenrod')
ax3.plot()
ax3.fill_between(t1, f1-e1, f1+e1, color = 'mediumpurple', alpha = 0.4)
ax3.fill_between(t1, f_h-e_h, f_h+e_h, color = 'darkorchid', alpha = 0.3)
ax3.fill_between(t2, f2-e2, f2+e2, color = 'khaki', alpha = 0.4)
ax3.fill_between(t2, f_b-e_b, f_b+e_b, color = 'goldenrod', alpha = 0.3)
ax3.legend()
ax3.set_ylabel('folding fraction')
ax3.set_xlabel('temp (K)')
ax3.set_xlim([270., 375.])
ax3.set_ylim([0., 1.])
prune(ax3, nbins = 6) 

# contact map
ax4 = axs[3]
cmap = np.loadtxt(os.path.join(DIR, 'leu15_6.280_cmap.txt'))
cmap = np.transpose(cmap)
nres = len(cmap)
im4 = ax4.contourf(cmap, origin = 'lower', interpolation = 'none', aspect = 'auto',
                  extent = [-1, nres+1, -1, nres+1], cmap = cm.binary)
ax4.set_xlabel('residue')
ax4.set_ylabel('residue')
# annotate the chains with brackets
ax4R = ax4.twinx()
bracklen = 1.5
for i in range(6):
    ax4.annotate(string.ascii_uppercase[i], xy = (0.09 * (1.8*i+1), 0.02), xycoords = 'axes fraction', 
                 bbox = dict(boxstyle = 'square', fc = 'white'),
                 ha = 'center', va = 'bottom', fontsize = 8,
                 arrowprops = dict(arrowstyle = '-[, widthB = 1.25, lengthB = 0.5', lw = 1.5))
    
    ax4R.annotate(string.ascii_uppercase[i], 
                 xy = (1.02, 0.09 * (1.8*i+1)), xycoords = 'axes fraction',
                 bbox = dict(boxstyle = 'square', fc = 'white'),
                 ha = 'left', va = 'center', fontsize = 8,
                 arrowprops = dict(arrowstyle = '-[, widthB = 1.1, lengthB = 0.5', lw = 1.5))
    
prune(ax4, nbins = 7)
ax4R.set_yticks([])
ax4R.set_yticklabels([])

# put panel labels
for n, ax in enumerate(axs):
    txt = '(%s)' % string.ascii_uppercase[n]
    ax.text(0, 1.05, txt, transform = ax.transAxes, weight = 'bold')
    
fig.tight_layout()
savefig(fig, 'fig6')
plt.close()

/share/apps/python/anaconda-python/lib/python2.7/site-packages/ipykernel/__main__.py:31: RuntimeWarning: divide by zero encountered in log


#### restore matplotlib params

In [332]:
matplotlib.rcParams.update(PARAMS)

# Native and non-native potentials

In [299]:
def extend_potential(r, p):
    dr = r[1] - r[0]
    r_last = r[-1]
    p_last = p[-1]
    r_extra = np.arange(r_last, 6., dr)
    p_extra = np.ones(len(r_extra)) * p_last
    r_new = np.append(r, r_extra)
    p_new = np.append(p, p_extra)
    p_new -= p_new[-1]
    kt = kB * 294.84
    p_new /= kt
    return r_new, p_new
    
data_nn = np.loadtxt('DATA/POTENTIALS/nonnative.dat')
r_nn = data_nn[:,1]
p_nn = data_nn[:,2]
r_nn, p_nn = extend_potential(r_nn, p_nn)

fig = plt.figure(figsize = (set_width('single') * 1.6, 4.5))
ax = fig.add_subplot(1,1,1)
# plot native
for i, ff in enumerate(fftypes):
    data_n = np.loadtxt('DATA/POTENTIALS/%s_native.dat' % ff)
    r_n = data_n[:,1]
    p_n = data_n[:,2]
    r_n, p_n = extend_potential(r_n, p_n)
    ax.plot(r_n, p_n, color = ffclrs[i], label = fflabels[i])

# plot non-native
ax.plot(r_nn, p_nn, color = 'black', label = 'non-native')

ax.set_xlim([2.5, 12])
ax.set_ylim([-11, 8])
ax.axhline(0., ls = '--', lw = 1, color = 'black')
ax.legend()
ax.set_xlabel(r'$r (\AA)$')
ax.set_ylabel(r'$u(r)$' + ' ' + r'$(k_B T)$')
prune(ax, nbins = 7)
    
fig.tight_layout()
savefig(fig, 'fig7')
plt.close()

# RMSD comparisons

#### version-1

In [157]:
with open('DATA/EXPERIMENTS/rmsd_zin.dat') as of: lines = of.readlines()
lines = lines[2:]
protein_type_labels = [r'$\alpha$', r'$\beta$', r'$\alpha+\beta$']
classlabels = (r'$RMSD <= 2\AA$', r'$RMSD \in (2, 4) \AA $', r'$RMSD >= 4\AA$')
nfftypes = len(fftypes)
nclasses = 3
classclrs = ['lightcoral', 'lightskyblue', 'yellowgreen']
data = {'alpha': np.zeros([nfftypes, nclasses]), 
        'beta': np.zeros([nfftypes, nclasses]), 
        'alpha_beta': np.zeros([nfftypes, nclasses])}
classind = None
for line in lines:
    l = line.split()
    this_protein_type = l[0]
    rmsd = [float(l[i]) for i in range(2,6)]
    for i, x in enumerate(rmsd):
        if x <= 2: classind = 0
        elif x > 2 and x < 4: classind = 1
        else: classind = 2
        data[this_protein_type][i, classind] += 1

    
fig = plt.figure(figsize = (set_width('single'), 4))
axs = []
axRs = []

# convert to fractions
for k, v in data.iteritems():
    for i in range(nfftypes):
        v[i,:] /= np.sum(v[i, :])
    data[k] = np.nan_to_num(v)

index = np.arange(nclasses)
bar_width = 0.1

for i, protein_type in enumerate(['alpha', 'beta', 'alpha_beta']):
    ax = fig.add_subplot(3, 1, i+1)
    axR = ax.twinx()
    for j in range(nfftypes):
        rects = ax.bar(index + (j+1)*bar_width, data[protein_type][j,:], bar_width, 
                       align = 'center', color = ffclrs[j], label = fflabels[j])
    if i == 0: ax.legend(prop = {'size': 7.2})
    prune(ax, doX = False)
    ax.set_ylim([0, 1.05])
    
    ax.yaxis.set_major_formatter(FormatStrFormatter('%1.1f'))
    ax.xaxis.set_ticks([])
    ax.set_xticklabels([])
    
    axR.xaxis.set_ticks([])
    axR.set_xticklabels([])
    axR.yaxis.set_ticks([])
    axR.set_yticklabels([])
    axR.set_ylabel(protein_type_labels[i], fontsize = 15)
    
    axs.append(ax)
    axRs.append(axR)

ax2 = axs[-2]
ax2.set_ylabel('fraction of sequences', fontsize = 12)
    
ax3 = axs[-1]
plt.xticks(index + bar_width * 2.5, classlabels)

plt.subplots_adjust(hspace = 0.0)
savefig(fig, 'fig8')
plt.close()

#### version 2 (stacked chart)

In [294]:
with open('DATA/EXPERIMENTS/rmsd_zin.dat') as of: lines = of.readlines()
lines = lines[2:]
protein_type_labels = [r'$\alpha$', r'$\beta$', r'$\alpha+\beta$']
classlabels = (r'$RMSD <= 2\AA$', r'$RMSD \in (2, 4) \AA $', r'$RMSD >= 4\AA$')
nfftypes = len(fftypes)
nclasses = 3
classclrs = ['mediumseagreen', 'teal', 'rosybrown']
data = {'alpha': np.zeros([nfftypes, nclasses]), 
        'beta': np.zeros([nfftypes, nclasses]), 
        'alpha_beta': np.zeros([nfftypes, nclasses])}
classind = None
for line in lines:
    l = line.split()
    this_protein_type = l[0]
    rmsd = [float(l[i]) for i in range(2,6)]
    for i, x in enumerate(rmsd):
        if x <= 2: classind = 0
        elif x > 2 and x < 4: classind = 1
        else: classind = 2
        data[this_protein_type][i, classind] += 1

# convert to fractions
for k, v in data.iteritems():
    for i in range(nfftypes):
        v[i,:] /= np.sum(v[i, :])
    data[k] = np.nan_to_num(v)

    
fig = plt.figure(figsize = (set_width('single') * 1.1, 4))
axs = []
axRs = []
fflabels[2] += r'$\hspace{5pt}$'

index = np.arange(nfftypes)
bar_width = 0.5

for i, protein_type in enumerate(['alpha', 'beta', 'alpha_beta']):
    ax = fig.add_subplot(3, 1, i+1)
    axR = ax.twinx()
    for j in range(nclasses):
        label = classlabels[j]
        bottom = np.sum(data[protein_type][:, 0:j], axis = 1)
        rects = ax.bar(index, data[protein_type][:, j], bar_width, 
                       align = 'center', color = classclrs[j], label = label,
                       bottom = bottom, edgecolor = 'black')
    
    prune(ax, doX = False)
    ax.set_ylim([0, 1.2])
    ax.yaxis.set_major_formatter(FormatStrFormatter('%1.1f'))
    ax.xaxis.set_ticks([])
    ax.set_xticklabels([])
    
    axR.xaxis.set_ticks([])
    axR.set_xticklabels([])
    axR.yaxis.set_ticks([])
    axR.set_yticklabels([])
    axR.set_ylabel(protein_type_labels[i])
    
    axs.append(ax)
    axRs.append(axR)
    
ax2 = axs[-2]
ax2.set_ylabel('fraction of sequences')
    
ax3 = axs[-1]
plt.xticks(index, fflabels)

# put legend below plot
ax.legend(loc = 'upper center', bbox_to_anchor = (0.5, -0.2), ncol = 3,
         prop = {'size': 10})

plt.subplots_adjust(hspace = 0.0)
savefig(fig, 'fig8_stacked')
plt.close()

# experiments

#### native-predicted overlaps

In [274]:
def get_overlap(target_set_type, fftype, figname):
    # get rmsds
    with open('DATA/EXPERIMENTS/rmsd_zin.dat') as of: lines = of.readlines()
    lines = lines[2:]
    rmsds = []
    if target_set_type == 'small': lines = lines[:10]
    else: lines = lines[10:]
    for line in lines:
        if fftype == 'leu15': col = 2
        if fftype == 'val15': col = 3
        if fftype == 'leu_val_heteropoly_360': col = 4
        if fftype == 'leu_val_ex_300': col = 5
        this_rmsd = float(line.split()[col])
        rmsds.append(this_rmsd)
    # fig settings
    if target_set_type == 'small':
        keys = small_set
        labels = small_set_labels
        nrows = 2
        ncols = 5
        fig_height = 5
        fig_width = set_width('dbl') * 1.5
    if target_set_type == 'large':
        keys = large_set
        labels = large_set_labels
        nrows = 3
        ncols = 3
        fig_height = 7.8
        fig_width = set_width('dbl') * 1.5
    
    ImgDir = '%s_%s_set' % (fftype, target_set_type)
    ImgDir = os.path.join('DATA', 'EXPERIMENTS', ImgDir)
    
    fig = plt.figure(figsize = (fig_width, fig_height))
    axs = []
    ax_legend = None
    for i in range(nrows*ncols):
        ax = fig.add_subplot(nrows, ncols, i+1)
        imgfile = os.path.join(ImgDir, keys[i] + '_overlay.png')
        pic = mpimg.imread(imgfile)
        ax.imshow(pic, aspect = 'auto')
        title = labels[i] + r'$, %1.1f \mathring{A}$' % rmsds[i]
        ax.set_title(title)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        axs.append(ax)
        legend_test1 = target_set_type == 'small' and keys[i] == '1gb1'
        legend_test2 = target_set_type == 'large' and keys[i] == '1shg'
        if legend_test1 or legend_test2: ax_legend = ax
    
    # put legend below plot
    native = mpatches.Patch(color = 'blue', label = 'native')
    predicted = mpatches.Patch(color = 'red', label = 'predicted')
    ax_legend.legend(handles = [native, predicted], loc = 'upper center', bbox_to_anchor = (0.5, 0.05), 
                     ncol = 2, prop = {'size': 16})
    fig.tight_layout()
    savefig(fig, figname)
    plt.close()
    return

get_overlap('small', 'leu_val_ex_300', 'fig9')
get_overlap('large', 'leu_val_ex_300', 'fig10')

#### large proteins

In [258]:
rmsd = {'1fue': 1.8, '8tim': 2.3}
filenames = {'1fue': 'DATA/LARGE_PROTEIN/overlay_1fue.png',
             '8tim': 'DATA/LARGE_PROTEIN/overlay_8tim.png'}
titles = {'1fue' : '(bacterial) Flavodoxin' + r'$, %1.1f \mathring{A}$' % rmsd['1fue'],
          '8tim' : ' TIM barrel' + r'$, %1.1f \mathring{A}$' % rmsd['8tim']
         }

fig = plt.figure(figsize = (set_width('single') * 1.5, 2.5))

ax1 = fig.add_subplot(1,2,1)
pic1 = mpimg.imread(filenames['1fue'])
ax1.imshow(pic1, aspect = 'auto')
ax1.set_title(titles['1fue'], fontsize = 14)

ax2 = fig.add_subplot(1,2,2)
pic2 = mpimg.imread(filenames['8tim'])
ax2.imshow(pic2, aspect = 'auto')
ax2.set_title(titles['8tim'], fontsize = 14)

for ax in [ax1, ax2]:
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

native = mpatches.Patch(color = 'blue', label = 'native')
predicted = mpatches.Patch(color = 'red', label = 'predicted')
ax1.legend(handles = [native, predicted], loc = 'upper center', bbox_to_anchor = (1.2, 0.05), 
            ncol = 2)
fig.tight_layout()
savefig(fig, 'fig11')
plt.close()

#### deleting native contacts

In [261]:
Dir = 'DATA/ERODENATIVE/leu_val_ex_300_1pgb_erodenative'
fracs = [0.00, 0.05, 0.10, 0.15, 0.20]
rmsds = np.loadtxt(os.path.join(Dir, 'rmsds.txt'))
fig = plt.figure(figsize = (set_width('dbl') * 1.8, 6))
axs = []
ax_legend = None
for i, frac in enumerate(fracs):
    filename = os.path.join(Dir, 'f_%1.2f.png' % frac)
    pic = mpimg.imread(filename)
    rmsd = rmsds[i,1]
    title = '%d\%% removed\n' % (100 * frac)
    title += '$ \mathrm{RMSD} = %1.1f \mathring{A}$' % rmsd
    ax = fig.add_subplot(1, len(fracs), i+1)
    ax.imshow(pic)
    ax.set_xticks([]) ; ax.set_yticks([])
    ax.set_xticklabels([]) ; ax.set_yticklabels([])
    ax.set_title(title)
    axs.append(ax)
    if i == 2: ax_legend = ax

native = mpatches.Patch(color = 'blue', label = 'native')
predicted = mpatches.Patch(color = 'red', label = 'predicted')
ax_legend.legend(handles = [native, predicted], loc = 'upper center', bbox_to_anchor = (0.5, 0.08), 
                 ncol = 2)
plt.subplots_adjust(wspace = 0.05)
savefig(fig, 'fig12')
plt.close()